In [1]:
import dwavebinarycsp
import dimod
import neal
import numpy as np
from dwavebinarycsp.core.constraint import Constraint
from dwave.system import DWaveSampler, EmbeddingComposite

# Solve a binary linear Equation Ax=b as csp with default Solver Advantage_system1.1
Problem ID c3d819e3-f75f-41d2-be4d-d4552c85dbb1

In [2]:
A = np.array([[1,1,1,1,1],
              [2,2,2,2,2],
              [0,1,2,3,4],
              [2,1,4,1,0],
              [0,1,0,1,0]])
b = np.array([2, 4, 3, 5, 1])
solution = {'x0':0, 'x1':1,'x2':1,'x3':0,'x4':0 }
no_solution = {'x0':0, 'x1':1,'x2':1,'x3':1,'x4':1 }

In [3]:
m, n = A.shape
vartype = dimod.Vartype.BINARY
csp = dwavebinarycsp.ConstraintSatisfactionProblem(vartype)
x = tuple(['x' + str(j) for j in range(n)])

def make_f(i): # avoid late binding of i in for loop
    return lambda *z : (sum(A[i,j]*z[j] for j in range(n)) == b[i])

for v in x:
    csp.add_variable(v)
for i in range(m):
    constraint = Constraint.from_func(make_f(i),x,vartype)
    csp.add_constraint(constraint)

In [4]:
def printAndGetTestResults(sampler, bqm):
    sampleset = sampler.sample(bqm)
    print("Sampling by ", sampler)
    for sample, energy in sampleset.data(['sample', 'energy']):
        isValid = csp.check(sample)
        print(sample, isValid, energy)
        if not isValid : return sampleset

In [5]:
bqm = dwavebinarycsp.stitch(csp)
samplers = {"BruteForce" : dimod.ExactSolver() , "Neal" : neal.SimulatedAnnealingSampler()}
samplesets = {s: printAndGetTestResults(samplers[s],bqm) for s in samplers.keys()}

Sampling by  <dimod.reference.samplers.exact_solver.ExactSolver object at 0x7f9463ee2a90>
{'x0': 0, 'x1': 1, 'x2': 1, 'x3': 0, 'x4': 0} True 3.5840130863107333e-10
{'x0': 0, 'x1': 0, 'x2': 1, 'x3': 1, 'x4': 0} False 4.00000000017501
Sampling by  <neal.sampler.SimulatedAnnealingSampler object at 0x7f9463ee2640>
{'x0': 0, 'x1': 1, 'x2': 1, 'x3': 0, 'x4': 0} True 3.5840486134475213e-10


In [7]:
sampler = EmbeddingComposite(DWaveSampler())
sampleset = sampler.sample(bqm,num_reads = 100)

In [9]:
sampleset.to_pandas_dataframe()

,x0,x1,x2,x3,x4,chain_break_fraction,energy,num_occurrences
0,0,1,1,0,0,0.0,3.584013e-10,69
1,0,0,1,1,0,0.0,4.000000e+00,12
2,1,0,0,1,0,0.0,4.194090e+00,14
3,0,0,0,1,0,0.0,7.548199e+00,2
4,0,1,1,0,1,0.0,8.000000e+00,1
5,1,0,0,1,1,0.0,1.219409e+01,2
